Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [11]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    print (conv.get_shape)
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    print (conv.get_shape)
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print (hidden.get_shape)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

<bound method Tensor.get_shape of <tf.Tensor 'Conv2D:0' shape=(16, 14, 14, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Conv2D_1:0' shape=(16, 7, 7, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Relu_2:0' shape=(16, 64) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Conv2D_2:0' shape=(10000, 14, 14, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Conv2D_3:0' shape=(10000, 7, 7, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Relu_5:0' shape=(10000, 64) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Conv2D_4:0' shape=(10000, 14, 14, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Conv2D_5:0' shape=(10000, 7, 7, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Relu_8:0' shape=(10000, 64) dtype=float32>>


In [12]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.398316
Minibatch accuracy: 6.2%
Validation accuracy: 9.9%
Minibatch loss at step 50: 1.747002
Minibatch accuracy: 50.0%
Validation accuracy: 45.3%
Minibatch loss at step 100: 1.033177
Minibatch accuracy: 56.2%
Validation accuracy: 66.8%
Minibatch loss at step 150: 0.884526
Minibatch accuracy: 81.2%
Validation accuracy: 70.9%
Minibatch loss at step 200: 0.720784
Minibatch accuracy: 75.0%
Validation accuracy: 75.9%
Minibatch loss at step 250: 0.913647
Minibatch accuracy: 81.2%
Validation accuracy: 77.5%
Minibatch loss at step 300: 0.467607
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Minibatch loss at step 350: 0.631119
Minibatch accuracy: 75.0%
Validation accuracy: 79.0%
Minibatch loss at step 400: 0.778026
Minibatch accuracy: 87.5%
Validation accuracy: 78.9%
Minibatch loss at step 450: 0.532508
Minibatch accuracy: 87.5%
Validation accuracy: 79.2%
Minibatch loss at step 500: 0.299076
Minibatch accuracy: 87.5%
Validation accuracy: 81.1%
Min

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():
  keep_prob = tf.placeholder(tf.float32)
  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data, dropout):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    print (hidden.get_shape)
    h_pool1 = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    print (h_pool1.get_shape)
    conv = tf.nn.conv2d(h_pool1, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    print (hidden.get_shape)
    h_pool2 = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 
    print (h_pool2.get_shape)
    shape = h_pool2.get_shape().as_list()
    reshape = tf.reshape(h_pool2, [shape[0], shape[1] * shape[2] * shape[3]])
    h_fc1 = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    print (h_fc1.get_shape)
    h_fc1_drop = tf.nn.dropout(h_fc1, dropout)    
    return tf.matmul(h_fc1_drop, layer4_weights) + layer4_biases
    
  
  # Training computation.
  logits = model(tf_train_dataset, keep_prob)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, 1.0))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, 1.0))

<bound method Tensor.get_shape of <tf.Tensor 'Relu:0' shape=(16, 28, 28, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'MaxPool:0' shape=(16, 14, 14, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Relu_1:0' shape=(16, 14, 14, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'MaxPool_1:0' shape=(16, 7, 7, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Relu_2:0' shape=(16, 64) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Relu_3:0' shape=(10000, 28, 28, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'MaxPool_2:0' shape=(10000, 14, 14, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Relu_4:0' shape=(10000, 14, 14, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'MaxPool_3:0' shape=(10000, 7, 7, 16) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'Relu_5:0' shape=(10000, 64) dtype=float32>>
<bound method Tensor.get_shape of <tf.Tensor 'R

In [6]:
num_steps = 95000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 1.0}
      l, predictions = session.run([loss, train_prediction], feed_dict=feed_dict)        
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.216029
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.244316
Minibatch accuracy: 18.8%
Validation accuracy: 16.6%
Minibatch loss at step 100: 1.918412
Minibatch accuracy: 50.0%



KeyboardInterrupt



---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [6]:
batch_size = 16
num_channels = 1

c1_depth = 6
c1_ker_sz = 5
c3_depth = 16
c3_ker_sz = 6
c5_depth = 128
c5_ker_sz = 6

num_hidden = 64

graph = tf.Graph()
with graph.as_default():
 # Input data.
  tf_train_dataset = tf.placeholder(
  tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
# Variables.
  c1_weights = tf.Variable(tf.truncated_normal(
    [c1_ker_sz, c1_ker_sz, num_channels, c1_depth], stddev=0.1))
  c1_biases = tf.Variable(tf.zeros([c1_depth]))
  c3_weights = tf.Variable(tf.truncated_normal(
    [c3_ker_sz, c3_ker_sz, c1_depth, c3_depth], stddev=0.1))
  c3_biases = tf.Variable(tf.constant(1.0, shape=[c3_depth]))
  c5_weights = tf.Variable(tf.truncated_normal(
    [c5_ker_sz, c5_ker_sz, c3_depth, c5_depth], stddev=0.1))
  c5_biases = tf.Variable(tf.constant(1.0, shape=[c5_depth]))
  c5_conv_dim = (((((image_size+1)//2) + 1) // 2) + 1 )//2
  fc_weights = tf.Variable(tf.truncated_normal(
    [c5_conv_dim * c5_conv_dim * c5_depth, num_hidden], stddev=0.1))
  fc_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  out_weights = tf.Variable(tf.truncated_normal(
    [num_hidden, num_labels], stddev=0.1))
  out_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))

# Model.
  def model(data):
        print(data.get_shape().as_list())
        conv = tf.nn.conv2d(data, c1_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + c1_biases)
        print(conv.get_shape().as_list())
        pooled = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        print(pooled.get_shape().as_list())
        conv = tf.nn.conv2d(pooled, c3_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + c3_biases)
        pooled = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        shape = pooled.get_shape().as_list()
        print(shape)
        conv = tf.nn.conv2d(pooled, c5_weights, [1, 1, 1, 1], padding='SAME')
        hidden = tf.nn.relu(conv + c5_biases)
        pooled = tf.nn.max_pool(hidden, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        shape = pooled.get_shape().as_list()
        print(shape)
        reshape = tf.reshape(pooled, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, fc_weights) + fc_biases)
        return tf.matmul(hidden, out_weights) + out_biases

# Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

# Optimizer.
#optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  optimizer = tf.train.AdagradOptimizer(0.05).minimize(loss)

# Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

[16, 28, 28, 1]
[16, 28, 28, 6]
[16, 14, 14, 6]
[16, 7, 7, 16]
[16, 4, 4, 128]
[10000, 28, 28, 1]
[10000, 28, 28, 6]
[10000, 14, 14, 6]
[10000, 7, 7, 16]
[10000, 4, 4, 128]
[10000, 28, 28, 1]
[10000, 28, 28, 6]
[10000, 14, 14, 6]
[10000, 7, 7, 16]
[10000, 4, 4, 128]


In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6.906130
Minibatch accuracy: 18.8%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.140933
Minibatch accuracy: 25.0%
Validation accuracy: 25.6%
Minibatch loss at step 100: 1.516758
Minibatch accuracy: 43.8%
Validation accuracy: 57.6%
Minibatch loss at step 150: 0.590568
Minibatch accuracy: 81.2%
Validation accuracy: 67.7%
Minibatch loss at step 200: 0.567323
Minibatch accuracy: 81.2%
Validation accuracy: 77.6%
Minibatch loss at step 250: 1.125589
Minibatch accuracy: 68.8%
Validation accuracy: 77.1%
Minibatch loss at step 300: 0.467870
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 350: 0.636360
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 400: 0.581453
Minibatch accuracy: 87.5%
Validation accuracy: 81.2%
Minibatch loss at step 450: 0.397667
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
Minibatch loss at step 500: 0.255166
Minibatch accuracy: 93.8%
Validation accuracy: 82.8%
M